In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [101]:
lambda_1 = 100
lambda_2 = 1
alpha = 1.0
delta = 1e-3
rho = 50
max_iter = 100
Z_inner_iter = 10
W_lr = None  # 不需要lr, W有闭式解
Z_lr = 1e-3

In [4]:
load_path = "/data/user/tourist/mixed-percision-quantization-for-SwinIR/scripts/x.pt"
x = torch.load(load_path)
x.shape
x = x.reshape(-1, x.shape[-1])
x.shape

/tmp/ipykernel_1926056/350791921.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = torch.load(load_path)


torch.Size([131072, 60])

In [6]:
loadd_path = "/data/user/tourist/mixed-percision-quantization-for-SwinIR/scripts/weight.pt"
weight = torch.load(loadd_path)
weight = weight.T
weight = weight[:, :60]

weight.shape

/tmp/ipykernel_1926056/4040728203.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weight = torch.load(loadd_path)


torch.Size([60, 60])

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
X_reshape = x.to(device)
Y_reshape = X_reshape@weight.to(device)
Y_reshape.shape

torch.Size([131072, 60])

In [8]:
U_, S_, Vh_ = torch.linalg.svd(weight, full_matrices=False)
alpha = torch.mean(S_)
print(alpha)
S_


tensor(0.9236, device='cuda:0', grad_fn=<MeanBackward0>)


tensor([4.1079, 3.9695, 3.4316, 3.0956, 3.0265, 2.8834, 2.8323, 2.8125, 2.4913,
        2.4582, 2.3853, 2.2984, 1.9307, 1.8277, 1.5488, 1.4096, 1.3564, 1.1351,
        0.9633, 0.8095, 0.7105, 0.6020, 0.5783, 0.5202, 0.4558, 0.4431, 0.3887,
        0.3753, 0.3493, 0.3165, 0.3075, 0.2783, 0.2654, 0.2466, 0.2341, 0.2206,
        0.2056, 0.1960, 0.1908, 0.1823, 0.1719, 0.1472, 0.1340, 0.1323, 0.1262,
        0.1165, 0.1066, 0.1001, 0.0892, 0.0781, 0.0694, 0.0629, 0.0525, 0.0481,
        0.0400, 0.0337, 0.0252, 0.0233, 0.0130, 0.0062], device='cuda:0',
       grad_fn=<LinalgSvdBackward0>)

In [142]:

#W = torch.zeros(weight.shape, device=device)
W = weight.clone().detach()
Z = W.clone().detach()
U = torch.zeros_like(W)

XTX = X_reshape.t() @ X_reshape
I_n = torch.eye(X_reshape.shape[1], device=device)
A = XTX + rho*I_n
A_inv = torch.inverse(A)
XTY = X_reshape.t() @ Y_reshape

In [141]:
U_, S_, Vh_ = torch.linalg.svd(X_reshape, full_matrices=False)
alpha = torch.mean(S_)
print(alpha)
S_


tensor(65.8947, device='cuda:0')


tensor([9.2088e+02, 7.2107e+02, 5.8632e+02, 3.2860e+02, 1.8841e+02, 1.8575e+02,
        1.6451e+02, 1.1080e+02, 1.0070e+02, 8.8202e+01, 7.4748e+01, 7.0583e+01,
        5.9892e+01, 5.9566e+01, 5.2480e+01, 4.9497e+01, 3.3651e+01, 3.3322e+01,
        2.8065e+01, 2.4262e+01, 1.4341e+01, 1.1784e+01, 9.7186e+00, 8.1706e+00,
        7.4338e+00, 6.4441e+00, 5.2315e+00, 4.4822e+00, 4.1969e+00, 5.6869e-01,
        4.8679e-05, 4.2400e-05, 3.9042e-05, 3.6825e-05, 3.2521e-05, 3.1018e-05,
        2.9560e-05, 2.9429e-05, 2.8422e-05, 2.6960e-05, 2.6665e-05, 2.5685e-05,
        2.4636e-05, 2.4361e-05, 2.4240e-05, 2.3886e-05, 2.3678e-05, 2.3378e-05,
        2.2465e-05, 2.2207e-05, 2.1870e-05, 2.1510e-05, 2.1066e-05, 2.0858e-05,
        2.0282e-05, 1.9446e-05, 1.8727e-05, 1.7344e-05, 7.8976e-06, 6.5528e-06],
       device='cuda:0')

In [155]:
r = torch.tensor(60)
r_10 = int(torch.ceil(0.1*r).item())
r_20 = int(torch.ceil(0.2*r).item())
r_80 = int(torch.ceil(0.8*r).item())
r_90 = int(torch.ceil(0.9*r).item())
w = torch.ones(r)
# 前10%
w[:r_10] = 100
# 前10%-20%
w[r_10:r_20] = 1
# 中间60%已经是1了
# 后20%-10%
w[r_80:r_90] = 1
# 最后10%
w[r_90:] = 1
#w = torch.ones(r)

w = w.to(device)
for it in range(101):
    # W-update
    RHS = XTY + rho*(Z - U)
    W = A_inv @ RHS

    # Z-update
    W_plus_U = W + U
    #W_plus_U = W
    P, sigma_W, Qt = torch.linalg.svd(W_plus_U, full_matrices=False)
    r = len(sigma_W)


    # Compute target singular value t
    #t = torch.mean(sigma_W)
    t = 2


    # Update singular values
    sigma_Z = (rho * sigma_W + lambda_1 * t * w) / (rho + lambda_1*w)
    sigma_Z = torch.maximum(sigma_Z, torch.tensor(0))

    # Reconstruct Z
    Z = P @ torch.diag(sigma_Z) @ Qt


    # U-update (scaled dual variable)
    U = U + (W - Z)

    # 检查收敛
    if it % 10 == 0:
        res = F.mse_loss(X_reshape @ W, Y_reshape)
        primal_res = torch.norm(W - Z, p='fro')
        print(f"res: {res.item():.6f}")
        print(f"Iter {it}, primal_res: {primal_res.item():.6f}")

res: 0.001256
Iter 0, primal_res: 5.821691
res: 0.001217
Iter 10, primal_res: 4.555047
res: 0.001222
Iter 20, primal_res: 4.497575
res: 0.001226
Iter 30, primal_res: 4.523546
res: 0.001231
Iter 40, primal_res: 4.531183
res: 0.001234
Iter 50, primal_res: 4.535338
res: 0.001239
Iter 60, primal_res: 4.537772
res: 0.001241
Iter 70, primal_res: 4.539563
res: 0.001248
Iter 80, primal_res: 4.541033
res: 0.001249
Iter 90, primal_res: 4.541514
res: 0.001251
Iter 100, primal_res: 4.541873


In [128]:
(rho * sigma_W + lambda_1 * t * w) / (rho + lambda_1*w)
print(rho * sigma_W[0] + lambda_1 * t * w[0], rho + lambda_1*w[0])
print(rho * sigma_W[0], lambda_1 * t * w[0])
print(sigma_W[0])
print(t)

tensor(598.4934, device='cuda:0', grad_fn=<AddBackward0>) tensor(150., device='cuda:0')
tensor(379.4328, device='cuda:0', grad_fn=<MulBackward0>) tensor(219.0606, device='cuda:0', grad_fn=<MulBackward0>)
tensor(7.5887, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(2.1906, device='cuda:0', grad_fn=<MeanBackward0>)


In [156]:
U_, S_, Vh_ = torch.linalg.svd(W, full_matrices=False)
S_

tensor([3.8087, 3.4180, 2.7875, 2.7127, 2.6968, 2.6278, 2.4946, 2.3875, 2.3811,
        2.2992, 2.2859, 2.2371, 2.0016, 2.0013, 2.0011, 2.0009, 2.0008, 2.0008,
        2.0007, 2.0005, 2.0005, 2.0004, 2.0004, 2.0003, 2.0003, 2.0002, 2.0002,
        2.0001, 2.0000, 2.0000, 1.9999, 1.9999, 1.9998, 1.9998, 1.9997, 1.9996,
        1.9995, 1.9995, 1.9993, 1.9992, 1.9991, 1.9989, 1.9939, 1.8920, 1.8567,
        1.7581, 1.7361, 1.6618, 1.6232, 1.5935, 1.5556, 1.5086, 1.2935, 1.2035,
        1.0730, 0.9908, 0.8997, 0.6623, 0.5918, 0.5269], device='cuda:0',
       grad_fn=<LinalgSvdBackward0>)

In [213]:

from matplotlib import pyplot as plt


def draw_plot(weight):
    #weight = torch.abs(weight)
    if weight.is_cuda:
        weight = weight.detach().cpu().numpy()
    else:
        weight = weight.numpy()

    input_channels = weight.shape[0]
    output_channels = weight.shape[1]

    X = np.arange(output_channels)
    Y = np.arange(input_channels)
    X, Y = np.meshgrid(X, Y)

    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(111, projection='3d')

    top_1_percent_threshold = np.percentile(weight, 99)
    # 创建颜色数组，初始化为蓝色，shape 为 (n, 4) 表示 RGBA 值
    colors = np.zeros((weight.size, 4))

    colors[:] = [0, 0, 1, 0.3]  # 蓝色

    top_1_indices = weight.ravel() >= top_1_percent_threshold
    colors[top_1_indices] = [1, 0, 0, 0.8]  # 红色

    ax.bar3d(X.ravel(), Y.ravel(), np.zeros_like(X.ravel()), 0.2, 0.2, weight.ravel(), color=colors)

    ax.set_xlabel('Channel')
    ax.set_ylabel('')
    ax.set_zlabel('Magnitude')

    ax.grid(True)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', bottom=False, top=False, labelleft=False)



    ax.text2D(0.05, 0.95, "X", transform=ax.transAxes, fontsize=15, weight='bold')
    plt.title(f'input_sampled', fontsize=12)


    plt.show()

In [ ]:
draw_plot(weight)
torch.min(weight)

In [ ]:
draw_plot(W)
torch.min(W)

In [ ]:
def logdet_grad(Z, lambda_1, delta):
    Q = Z.t() @ Z + delta*torch.eye(Z.shape[1], device=Z.device)
    Q_inv = torch.inverse(Q)
    return -2*lambda_1 * Z @ Q_inv

def sigma_penalty_grad(Z, lambda_2, alpha):
    U_, S_, Vh_ = torch.linalg.svd(Z, full_matrices=False)
    diff = S_ - alpha
    S_grad = 2 * diff
    return lambda_2 * (U_ @ torch.diag(S_grad) @ Vh_)

for it in range(81):
    # W-update
    RHS = XTY + rho*(Z - U)
    W = A_inv @ RHS

    # Z-update with inner iteration
    R = W + U
    for _ in range(Z_inner_iter):
        grad_logdet = logdet_grad(Z, lambda_1, delta)
        grad_sigma = sigma_penalty_grad(Z, lambda_2, alpha)
        grad_quad = rho*(Z - R)
        grad_Z = grad_logdet + grad_sigma + grad_quad
        Z = Z - Z_lr * grad_Z

    # U-update (scaled dual variable)
    U = U + (W - Z)

    # 检查收敛
    if it % 10 == 0:
        res = F.mse_loss(X_reshape @ W, Y_reshape)
        primal_res = torch.norm(W - Z, p='fro')
        print(f"res: {res.item():.6f}")
        print(f"Iter {it}, primal_res: {primal_res.item():.6f}")

In [ ]:

def prox_nuclear_norm(Z, tau):
    U_, S_, Vh_ = torch.linalg.svd(Z, full_matrices=False)
    S_thresholded = torch.clamp(S_ - tau, min=0)
    return U_ @ torch.diag(S_thresholded) @ Vh_

def logdet_grad(Z, delta, lambda_1):
    n = Z.shape[1]
    I_n = torch.eye(n, device=Z.device)
    Q = Z.t() @ Z + delta * I_n
    Q_inv = torch.inverse(Q)
    grad = -2 * lambda_1 * Z @ Q_inv
    return grad

def Z_subproblem_update(W, U, lambda_1, lambda_2, rho, Z_init, delta=1e-3, lr=1e-3, inner_iter=10):
    Z_ = Z_init.clone()
    R = W + U  
    I_n = torch.eye(Z_.shape[1], device=Z_.device)
    for _ in range(inner_iter):
        # grad from logdet
        grad_logdet = logdet_grad(Z_, delta, lambda_1)
        # grad from quadratic
        grad_quad = rho*(Z_ - R)
        grad = grad_logdet + grad_quad
        Z_temp = Z_ - lr * grad
        Z_ = prox_nuclear_norm(Z_temp, lr * lambda_2)
    return Z_

def conjugate_gradient(Av_func, b, x0=None, tol=1e-6, max_iter=1000, verbose=False):
    if b.dim() != 1:
        raise ValueError("b.dim() != 1")
    x = torch.zeros_like(b) if x0 is None else x0.clone()
    r = b - Av_func(x)
    p = r.clone()
    rr_old = torch.dot(r, r)
    initial_norm = torch.sqrt(rr_old)
    if initial_norm < tol:
        return x
    for i in range(max_iter):
        Ap = Av_func(p)
        pAp = torch.dot(p, Ap)
        if pAp <= 0:
            raise RuntimeError("p^T A p <= 0")
        alpha = rr_old / pAp
        x = x + alpha * p
        r = r - alpha * Ap
        rr_new = torch.dot(r, r)
        res_norm = torch.sqrt(rr_new)
        if verbose and i % 50 == 0:
            print(f"CG Iter {i}, Residual: {res_norm.item():.6f}")
        if res_norm < tol:
            if verbose:
                print(f"Converged at iter {i}, Residual: {res_norm.item():.6f}")
            break
        beta = rr_new / rr_old
        p = r + beta * p
        rr_old = rr_new
    return x

def solve_W_subproblem(x_inverse, X_mat, Y_mat, Z, U, rho, W_init, tol=1e-6, max_cg_iter=1000, verbose=False):
    #x batch* 64, 60 w 60, 60  [60, 180]    [60, 120] [120, 60]
    # Y batch * 64, 60 or 180
    n = 60
    shape_w = W_init.shape
    R = Z - U
    # b = X^T Y + rho R
    XTY = (X_mat.T @ Y_mat) 
    b_mat = XTY + rho * R
    b_flat = b_mat.flatten()

    def Av_func(W_flat):
        W_mat = W_flat.reshape(shape_w)
        XW = X_mat @ W_mat  
        XTXW = X_mat.T @ XW
        AW = XTXW + rho * W_mat
        return AW.flatten()

    W0_flat = W_init.flatten()
    #W_flat_sol = conjugate_gradient(Av_func, b_flat, W0_flat, tol=tol, max_iter=max_cg_iter, verbose=verbose)
    W_flat_sol = x_inverse @ b_mat
    W_sol = W_flat_sol.reshape(shape_w)
    return W_sol

W = W.to(device)
Z = Z.to(device)
U = U.to(device)

for it in range(max_iter):
    # W-update
    I_n = torch.eye(X_reshape.shape[1], device=X_reshape.device)
    x_inverse = torch.inverse(X_reshape.T@X_reshape + rho * I_n)
    W = solve_W_subproblem(x_inverse, X_reshape, Y_reshape, Z, U, rho, W, tol=W_cg_tol, max_cg_iter=W_cg_max_iter, verbose=False)

    # Z-update
    Z = Z_subproblem_update(W, U, lambda_1, lambda_2, rho, Z, delta=delta, lr=Z_lr, inner_iter=Z_inner_iter)

    # U-update
    U = U + (W - Z)

    primal_res = torch.norm(W - Z, p='fro')
    res = torch.norm(X_reshape @ W - Y_reshape, p='fro')
    if it % 10 == 0:
        print(f"Iter {it}, res: {res.item():.6f}")
        print(f"Iter {it}, primal_res: {primal_res.item():.6f}")

    if primal_res.item() < 1e-6:
        print("Converged!")
        break

print("Optimization Finished. Final primal residual:", primal_res.item())